In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette(["#f28e2b","#4e79a7","#7f7f7f"])

HOUR_CSV = "hour_level.csv"
DAYPART_STATUS_CSV = "daypart_status.csv"
PICKUP_GAP_CSV = "pickup_daypart_gap.csv"

hour = pd.read_csv(HOUR_CSV)
daypart_status = pd.read_csv(DAYPART_STATUS_CSV)
pickup_gap = pd.read_csv(PICKUP_GAP_CSV)

# Flexible column mapping based on what exists
cols = {c.lower(): c for c in hour.columns}
req_col  = cols.get("requests", cols.get("total_requests","requests"))
unf_col  = cols.get("unfulfilled", cols.get("unfulfilled_count","unfulfilled"))
hr_col   = cols.get("request_hour", cols.get("hour","request_hour"))
pct_col  = cols.get("unfulfilled_pct_dec", cols.get("unfulfilled_pct", None))

# Create the exact columns the plotting code expects
hour = hour.rename(columns={req_col:"requests", unf_col:"unfulfilled", hr_col:"request_hour"})
hour["request_hour"] = hour["request_hour"].astype(int)

# If percent not present, compute it
if pct_col and pct_col in hour.columns:
    # If value looks like 0–1 keep; if 0–100 convert
    v = hour[pct_col].astype(float)
    hour["unfulfilled_pct_dec"] = np.where(v.max()>1.5, v/100.0, v)
else:
    hour["unfulfilled_pct_dec"] = hour["unfulfilled"] / hour["requests"]

daypart_order = ["Early Morning","Morning","Afternoon","Evening","Night","Late Night"]
print("Loaded:", len(hour), len(daypart_status), len(pickup_gap))


Loaded: 24 18 12


In [3]:
total_requests = int(hour["requests"].sum())
unfulfilled_count = int(hour["unfulfilled"].sum())
unfulfilled_pct = float(unfulfilled_count / total_requests)

kpis = pd.DataFrame({
    "metric":["total_requests","unfulfilled_count","unfulfilled_pct"],
    "value":[total_requests, unfulfilled_count, round(unfulfilled_pct,4)]
})
kpis.to_csv("kpis_summary.csv", index=False)

print({"Total Requests": total_requests,
       "Unfulfilled (count)": unfulfilled_count,
       "Unfulfilled (%)": f"{unfulfilled_pct*100:.1f}%"})


{'Total Requests': 6745, 'Unfulfilled (count)': 3914, 'Unfulfilled (%)': '58.0%'}


In [4]:
fig, ax1 = plt.subplots(figsize=(10,4))
ax1.bar(hour["request_hour"], hour["requests"], color="#f28e2b", label="Requests")
ax1.set_xlabel("Hour"); ax1.set_ylabel("Requests")

ax2 = ax1.twinx()
ax2.plot(hour["request_hour"], hour["unfulfilled_pct_dec"]*100,
         color="#4e79a7", linewidth=2.4, label="Unfulfilled %")
ax2.set_ylabel("Unfulfilled %"); ax2.set_ylim(0,100)

fig.suptitle("Hourly Requests vs Unfulfilled %")
fig.tight_layout()
fig.savefig("hourly_requests_unfulfilled.png", dpi=220)
plt.close(fig)
print("Saved hourly_requests_unfulfilled.png")


Saved hourly_requests_unfulfilled.png


In [5]:
fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(data=pickup_gap, x="daypart", y="unfulfilled", hue="pickup_point",
            order=daypart_order, ax=ax)
ax.set_title("Pickup × Daypart: Unfulfilled")
ax.set_xlabel("Daypart"); ax.set_ylabel("Unfulfilled")
ax.legend(title=None, loc="upper right")
fig.tight_layout()
fig.savefig("pickup_daypart_unfulfilled.png", dpi=220)
plt.close(fig)
print("Saved pickup_daypart_unfulfilled.png")


Saved pickup_daypart_unfulfilled.png


In [6]:
pt = daypart_status.pivot_table(index="daypart", columns="status",
                                values="n", aggfunc="sum").fillna(0)
pt = pt.reindex(daypart_order)

colors = {"Cancelled":"#4e79a7","No Cars Available":"#f28e2b","Trip Completed":"#7f7f7f"}
fig, ax = plt.subplots(figsize=(8,4))
bottom = np.zeros(len(pt))
for col in ["Cancelled","No Cars Available","Trip Completed"]:
    ax.bar(pt.index, pt[col].values, bottom=bottom, color=colors[col], label=col)
    bottom += pt[col].values

ax.set_title("Daypart × Status (Counts)")
ax.set_xlabel("Daypart"); ax.set_ylabel("Count")
ax.legend(ncol=3, bbox_to_anchor=(0.5,-0.18), loc="upper center")
fig.tight_layout()
fig.savefig("daypart_status_counts.png", dpi=220, bbox_inches="tight")
plt.close(fig)
print("Saved daypart_status_counts.png")


Saved daypart_status_counts.png


In [7]:
from matplotlib.backends.backend_pdf import PdfPages
imgs = ["hourly_requests_unfulfilled.png",
        "pickup_daypart_unfulfilled.png",
        "daypart_status_counts.png"]
with PdfPages("pandas_dashboard_export.pdf") as pdf:
    for img in imgs:
        fig = plt.figure(figsize=(11.69,8.27))  # A4 landscape
        ax = fig.add_axes([0,0,1,1]); ax.axis("off")
        ax.imshow(plt.imread(img))
        pdf.savefig(fig, bbox_inches="tight"); plt.close(fig)
print("Saved pandas_dashboard_export.pdf")


Saved pandas_dashboard_export.pdf


In [8]:
# Final: KPIs + ranked insights (text-only)
total_requests = int(hour["requests"].sum())
unfulfilled_count = int(hour["unfulfilled"].sum())
unfulfilled_pct = unfulfilled_count / total_requests

by_hour = (hour.groupby("request_hour")
           .agg(req=("requests","sum"), unf=("unfulfilled","sum"))
           .assign(unf_pct=lambda d: d["unf"]/d["req"])
           .sort_values("unf_pct", ascending=False))

by_daypart = (pickup_gap.groupby(["daypart","pickup_point"])["unfulfilled"]
              .sum().reset_index()
              .sort_values("unfulfilled", ascending=False))

# Save KPIs for repo
pd.DataFrame({
    "metric":["total_requests","unfulfilled_count","unfulfilled_pct"],
    "value":[total_requests, unfulfilled_count, round(float(unfulfilled_pct),4)]
}).to_csv("kpis_summary.csv", index=False)

print("KPIs")
print(f"- Total Requests: {total_requests}")
print(f"- Unfulfilled (count): {unfulfilled_count}")
print(f"- Unfulfilled (%): {unfulfilled_pct*100:.1f}%")

print("\nTop hours by unfulfilled% (first 5):")
print((by_hour["unf_pct"].mul(100).round(1)).head(5).to_string())

print("\nPickup × Daypart (top 6 unfulfilled):")
print(by_daypart.head(6).to_string(index=False))


KPIs
- Total Requests: 6745
- Unfulfilled (count): 3914
- Unfulfilled (%): 58.0%

Top hours by unfulfilled% (first 5):
request_hour
1     70.6
21    68.4
18    67.8
20    67.3
19    64.9

Pickup × Daypart (top 6 unfulfilled):
      daypart pickup_point  unfulfilled
      Evening      Airport         1145
Early Morning         City          700
      Morning         City          651
        Night      Airport          421
   Late Night         City          214
    Afternoon         City          205
